<a href="https://colab.research.google.com/github/JaydenCruz2004/DSC4420-MiniHackathon1/blob/main/DSC4420_MiniHackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -Uq

In [ ]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jaydencruz824 (jaydencruz824-belmont-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

Step 1: Define a sweep

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import wandb
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time

In [ ]:
# Expanded sweep config
sweep_config = {
    'method': 'random',   # options: grid, random, bayes
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        # Learning rate search space
        'learning_rate': {
            'values': [0.1, 0.01, 0.001, 0.0005, 0.0001, 0.00005]
        },

        # Batch sizes
        'batch_size': {
            'values': [32, 64, 128, 256, 512]
        },

        # Optimizer
        'optimizer': {
            'values': ['sgd', 'adam', 'adamw', 'rmsprop']
        },

        # Weight decay (regularization)
        'weight_decay': {
            'values': [0.0, 1e-4, 1e-3, 1e-2]
        },

        # Dropout
        'dropout': {
            'values': [0.0, 0.2, 0.5]
        },

        # Activation function
        'activation': {
            'values': ['ReLU', 'LeakyReLU', 'GELU', 'SiLU', 'Tanh']
        },

        # Model type
        'model': {
            'values': ['MLP', 'CNN']
        },

        # For MLP only: number of layers and hidden size
        'num_layers': {
            'values': [1, 2, 3, 4]
        },
        'hidden_dim': {
            'values': [64, 128, 256, 512]
        }
    }
}


In [ ]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_loss'},
 'parameters': {'activation': {'values': ['ReLU',
                                          'LeakyReLU',
                                          'GELU',
                                          'SiLU',
                                          'Tanh']},
                'batch_size': {'values': [32, 64, 128, 256, 512]},
                'dropout': {'values': [0.0, 0.2, 0.5]},
                'hidden_dim': {'values': [64, 128, 256, 512]},
                'learning_rate': {'values': [0.1,
                                             0.01,
                                             0.001,
                                             0.0005,
                                             0.0001,
                                             5e-05]},
                'model': {'values': ['MLP', 'CNN']},
                'num_layers': {'values': [1, 2, 3, 4]},
                'optimizer': {'values': ['sgd', 'adam', 'adam

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: ld6do6ly
Sweep URL: https://wandb.ai/jaydencruz824-belmont-university/pytorch-sweeps-demo/sweeps/ld6do6ly


In [ ]:
# check that there's more than just the CPU available
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")
if device==torch.device('cpu'): print("You should probably restart this with a GPU. It will be slow otherwise.")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

Using device: cpu
You should probably restart this with a GPU. It will be slow otherwise.


In [ ]:
# Available datasets with their normalization parameters
DATASETS = {
    'MNIST': {
        'dataset_class': torchvision.datasets.MNIST,
        'mean': (0.1307,), 'std': (0.3081,),
        'description': '28x28 grayscale handwritten digits (0-9)'
    },
    'FashionMNIST': {
        'dataset_class': torchvision.datasets.FashionMNIST,
        'mean': (0.2860,), 'std': (0.3530,),
        'description': '28x28 grayscale fashion items (10 classes: shirt, shoe, etc.)'
    },
    'CIFAR10': {
        'dataset_class': torchvision.datasets.CIFAR10,
        'mean': (0.4914, 0.4822, 0.4465), 'std': (0.2023, 0.1994, 0.2010),
        'description': '32x32 color images (10 classes: planes, cars, etc.)'
    },
    'CIFAR100': {
        'dataset_class': torchvision.datasets.CIFAR100,
        'mean': (0.5071, 0.4867, 0.4408), 'std': (0.2675, 0.2565, 0.2761),
        'description': '32x32 color images (100 classes)'
    }
}

print("Available datasets:")
for name, info in DATASETS.items():
    print(f"  {name}: {info['description']}")


# Also let's let the user choose different activation functions, which will be implemented globally:
activations = { 'ReLU': nn.ReLU(), 'GELU': nn.GELU(),  'SiLU': nn.SiLU(), 'LeakyReLU': nn.LeakyReLU(0.01)}
print("\nAvailable activations:", activations.keys())

Available datasets:
  MNIST: 28x28 grayscale handwritten digits (0-9)
  FashionMNIST: 28x28 grayscale fashion items (10 classes: shirt, shoe, etc.)
  CIFAR10: 32x32 color images (10 classes: planes, cars, etc.)
  CIFAR100: 32x32 color images (100 classes)

Available activations: dict_keys(['ReLU', 'GELU', 'SiLU', 'LeakyReLU'])


In [ ]:
#### YOUR CHOICES for dataset & activation functions
DATASET_NAME = 'CIFAR10'  # Choose between 'MNIST', 'FashionMNIST', 'CIFAR10', or 'CIFAR100'
ACTIVATION_NAME = 'ReLU'  # Options: 'ReLU', 'GELU', 'SiLU', 'LeakyReLU'

ACTIVATION = activations[ACTIVATION_NAME]

print(f"\nUsing: {DATASET_NAME}")
print(f"Activation function: {ACTIVATION_NAME}")


Using: CIFAR10
Activation function: ReLU


In [ ]:
def get_dataset_info(dataset_name):
    """Get dataset configuration and sample to determine properties."""
    config = DATASETS[dataset_name]
    basic_transform = transforms.Compose([transforms.ToTensor()])
    temp_dataset = config['dataset_class'](root='./data', train=True, download=True, transform=basic_transform)
    sample_img, _ = temp_dataset[0]

    return {
        'config': config,
        'num_channels': sample_img.shape[0],
        'img_height': sample_img.shape[1],
        'img_width': sample_img.shape[2],
        'num_classes': len(temp_dataset.classes),
        'class_names': temp_dataset.classes
    }

dataset_info = get_dataset_info(DATASET_NAME)

print(f"\nDataset Properties:")
print(f"  Image size: {dataset_info['num_channels']}x{dataset_info['img_height']}x{dataset_info['img_width']}")
print(f"  Number of classes: {dataset_info['num_classes']}")
print(f"  Classes: {dataset_info['class_names'][:10]}...")  # Show first 10 classes

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(dataset_info['config']['mean'], dataset_info['config']['std'])
])

train_dataset = dataset_info['config']['dataset_class'](root='./data', train=True, download=False, transform=transform)
test_dataset = dataset_info['config']['dataset_class'](root='./data', train=False, download=False, transform=transform)

batch_size = 128 # could make this bigger
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\nDataset loaded:")
print(f"  Training samples: {len(train_dataset):,}")
print(f"  Test samples: {len(test_dataset):,}")

100%|██████████| 170M/170M [00:01<00:00, 105MB/s]



Dataset Properties:
  Image size: 3x32x32
  Number of classes: 10
  Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']...

Dataset loaded:
  Training samples: 50,000
  Test samples: 10,000


In [ ]:
def plot_samples(dataset, dataset_info, num_samples=8):
    """Visualize some sample images"""
    fig, axes = plt.subplots(2, 4, figsize=(10, 5))
    axes = axes.ravel()
    mean, std = dataset_info['config']['mean'], dataset_info['config']['std']

    for i in range(num_samples):
        img, label = dataset[i]

        if len(mean) == 1:  # Grayscale
            img_denorm = img.squeeze() * std[0] + mean[0]
            axes[i].imshow(img_denorm, cmap='gray')
        else:  # Color
            img_denorm = img.clone()
            for c in range(len(mean)):
                img_denorm[c] = img_denorm[c] * std[c] + mean[c]
            img_denorm = torch.clamp(img_denorm, 0, 1).permute(1, 2, 0)  # (C,H,W) -> (H,W,C)
            axes[i].imshow(img_denorm)

        axes[i].set_title(f'Label: {label} ({dataset_info["class_names"][label]})')
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

plot_samples(train_dataset, dataset_info)

In [ ]:
class MLP(nn.Module):
    def __init__(self, dataset_info, hidden_sizes=[512, 256], dropout=0.2):
        super(MLP, self).__init__()

        input_size = (dataset_info['num_channels'] * dataset_info['img_height'] * dataset_info['img_width'])
        layers = []
        prev_size = input_size

        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                ACTIVATION,
                nn.Dropout(dropout)
            ])
            prev_size = hidden_size

        layers.append(nn.Linear(prev_size, dataset_info['num_classes']))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x.view(x.size(0), -1))  # Flatten: (batch, C, H, W) -> (batch, C*H*W)

mlp_model = MLP(dataset_info).to(device)
print(f"MLP Model Parameters: {sum(p.numel() for p in mlp_model.parameters()):,}")

In [ ]:
nn.Dropout(0.2)  # 0.2 = 20% dropout rate - randomly sets 20% of neurons to zero during training

In [ ]:
def train_model(model, train_loader, test_loader, epochs=50, lr=0.001):
    # Initialize wandb
    model_name = type(model).__name__
    wandb.init(project=f"convnet_{DATASET_NAME}", name=f"{model_name},{ACTIVATION_NAME}", reinit=True)
    wandb.config.update({
        "epochs": epochs,
        "batch_size": batch_size,
        "learning_rate": lr,
        "model": model_name,
        "optimizer": "Adam"
    })

    # Loss function:
    # Note: targets are just class indices (0-9), not one-hot vectors
    # nn.CrossEntropyLoss handles one-hot encoding internally for efficiency
    criterion = nn.CrossEntropyLoss()   # used for categorical variables, expects raw "logits"

    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    train_accuracies = []
    test_accuracies = []

    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        # Use tqdm for progress bar
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')

        for batch_idx, (data, target) in enumerate(pbar):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct_train += pred.eq(target.view_as(pred)).sum().item()
            total_train += target.size(0)

            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100.*correct_train/total_train:.2f}%'
            })

        # Calculate epoch metrics
        epoch_loss = running_loss / len(train_loader)
        train_acc = 100. * correct_train / total_train

        # Evaluation phase
        model.eval()
        test_loss = 0
        correct_test = 0
        total_test = 0

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                pred = output.argmax(dim=1, keepdim=True)
                correct_test += pred.eq(target.view_as(pred)).sum().item()
                total_test += target.size(0)

        test_acc = 100. * correct_test / total_test

        # Store metrics
        train_losses.append(epoch_loss)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)

        # Log to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": epoch_loss,
            "train_accuracy": train_acc,
            "test_accuracy": test_acc
        })

        print(f'Epoch {epoch+1}: Train Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%')

    wandb.finish()

    return {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'test_accuracies': test_accuracies,
        'final_test_acc': test_acc
    }

In [ ]:
print("Training MLP Model...")
start_time = time.time()
mlp_results = train_model(mlp_model, train_loader, test_loader, epochs=10)
mlp_training_time = time.time() - start_time
print(f"MLP Training completed in {mlp_training_time:.2f} seconds")

In [ ]:
class CNN(nn.Module):
    def __init__(self, dataset_info, base_channels=32, channel_mult=2, n_conv_layers=4):
        super(CNN, self).__init__()

        # Build conv layers dynamically
        self.conv_layers = nn.ModuleList()
        in_channels = dataset_info['num_channels']
        for i in range(n_conv_layers):
            out_channels = base_channels * (channel_mult ** i)
            self.conv_layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1))
            in_channels = out_channels

        self.activation = ACTIVATION
        self.dropout1 = nn.Dropout(0.25)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1) # put near end: yields one value per channel

        # Final channels after all conv layers
        final_channels = base_channels * (channel_mult ** (n_conv_layers - 1))
        self.fc = nn.Linear(final_channels, dataset_info['num_classes'])

    def forward(self, x):
        for conv_layer in self.conv_layers:
            x = self.activation(conv_layer(x))
            x = self.dropout1(x)
        x = self.global_avg_pool(x)  # one value per channel
        return self.fc(  x.flatten(start_dim=1) )  # flatten and run through linear layer



cnn_model = CNN(dataset_info).to(device)
print(f"CNN Model Parameters: {sum(p.numel() for p in cnn_model.parameters()):,}")

# quick test to make sure the forward() runs w/o errors:
rnd_batch = torch.randn([128,dataset_info['num_channels'], dataset_info['img_height'], dataset_info['img_width']]).to(device)
result = cnn_model(rnd_batch)
del rnd_batch

In [ ]:
print("\nTraining CNN Model...")
start_time = time.time()
cnn_results = train_model(cnn_model, train_loader, test_loader, epochs=10)
cnn_training_time = time.time() - start_time
print(f"CNN Training completed in {cnn_training_time:.2f} seconds")

In [ ]:
# Print comparison results
print("\n" + "="*60)
print("FINAL RESULTS COMPARISON")
print("="*60)
print(f"{'Metric':<20} {'MLP':<15} {'CNN':<15}")
print("-" * 60)
print(f"{'Parameters':<20} {sum(p.numel() for p in mlp_model.parameters()):,<15} {sum(p.numel() for p in cnn_model.parameters()):,<15}")
print(f"{'Training Time':<20} {mlp_training_time:.2f}s{'':<10} {cnn_training_time:.2f}s{'':<10}")
print(f"{'Final Test Acc':<20} {mlp_results['final_test_acc']:.2f}%{'':<10} {cnn_results['final_test_acc']:.2f}%{'':<10}")

# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Training loss comparison
epochs = range(1, 11)
ax1.plot(epochs, mlp_results['train_losses'], label='MLP', marker='o')
ax1.plot(epochs, cnn_results['train_losses'], label='CNN', marker='s')
ax1.set_title('Training Loss Comparison')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True)

# Test accuracy comparison
ax2.plot(epochs, mlp_results['test_accuracies'], label='MLP', marker='o')
ax2.plot(epochs, cnn_results['test_accuracies'], label='CNN', marker='s')
ax2.set_title('Test Accuracy Comparison')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def evaluate_model_detailed(model, test_loader, model_name):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)

            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)

    # Calculate per-class accuracy
    class_accuracies = []
    for i in range(10):
        mask = all_targets == i
        if mask.sum() > 0:
            class_acc = (all_preds[mask] == all_targets[mask]).mean() * 100
            class_accuracies.append(class_acc)
        else:
            class_accuracies.append(0)

    print(f"\n{model_name} - Per-class Accuracies:")
    for i, acc in enumerate(class_accuracies):
        print(f"Digit {i}: {acc:.2f}%")

    return all_preds, all_targets, class_accuracies

def plot_confusion_matrix(y_true, y_pred, model_name, class_names=None):
    """Plot confusion matrix with color-coded cells showing prediction counts."""
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y_true, y_pred)  # Calculate confusion matrix
    fig, ax = plt.subplots(figsize=(max(8, len(cm)), max(6, len(cm))))  # Create plot
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)  # Create heatmap
    plt.colorbar(im, ax=ax)  # Add colorbar

    if class_names is None:
        class_names = [str(i) for i in range(len(cm))]

    # For datasets with many classes, show only every nth label
    if len(class_names) > 20:
        step = len(class_names) // 10
        tick_labels = [class_names[i] if i % step == 0 else '' for i in range(len(class_names))]
        tick_positions = range(len(class_names))
    else:
        tick_labels = class_names
        tick_positions = range(len(class_names))

    ax.set_xticks(tick_positions)
    ax.set_yticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    ax.set_yticklabels(tick_labels)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")  # Rotate labels

    # Add text annotations with counts (but only if not too many classes)
    if len(cm) <= 20:
        thresh = cm.max() / 2.
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, format(cm[i, j], 'd'),
                       ha="center", va="center",
                       color="white" if cm[i, j] > thresh else "black",
                       fontsize=max(8, 12 - len(cm)//5))

    ax.set_ylabel('True Label', fontsize=14)
    ax.set_xlabel('Predicted Label', fontsize=14)
    ax.set_title(f'Confusion Matrix - {model_name}', fontsize=16, pad=20)

    correct, total = np.trace(cm), np.sum(cm)  # Calculate accuracy
    accuracy = correct / total * 100

    plt.figtext(0.02, 0.02, f'Overall Accuracy: {accuracy:.2f}%',
                fontsize=12, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
    plt.tight_layout()
    plt.show()

mlp_preds, mlp_targets, mlp_class_acc = evaluate_model_detailed(mlp_model, test_loader, "MLP")  # Evaluate models
cnn_preds, cnn_targets, cnn_class_acc = evaluate_model_detailed(cnn_model, test_loader, "CNN")

print("\nConfusion Matrix Analysis:")
plot_confusion_matrix(mlp_targets, mlp_preds, "MLP", class_names=dataset_info['class_names'])
plot_confusion_matrix(cnn_targets, cnn_preds, "CNN", class_names=dataset_info['class_names'])

In [ ]:
def show_predictions(model, test_dataset, dataset_info, model_name, num_samples=8):
    model.eval()
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    axes = axes.ravel()

    mean, std = dataset_info['config']['mean'], dataset_info['config']['std']
    indices = np.random.choice(len(test_dataset), num_samples, replace=False)

    with torch.no_grad():
        for i, idx in enumerate(indices):
            img, true_label = test_dataset[idx]
            img_tensor = img.unsqueeze(0).to(device)

            output = model(img_tensor)
            pred_label = output.argmax(dim=1).item()
            confidence = F.softmax(output, dim=1).max().item()

            # Denormalize image for display
            if len(mean) == 1:  # Grayscale
                display_img = img.squeeze() * std[0] + mean[0]
                axes[i].imshow(display_img, cmap='gray')
            else:  # RGB
                display_img = img.clone()
                for c in range(len(mean)):
                    display_img[c] = display_img[c] * std[c] + mean[c]
                display_img = torch.clamp(display_img, 0, 1).permute(1, 2, 0)
                axes[i].imshow(display_img)

            color = 'green' if pred_label == true_label else 'red'
            class_name = dataset_info['class_names'][true_label]
            pred_class_name = dataset_info['class_names'][pred_label]
            axes[i].set_title(f'True: {class_name}\nPred: {pred_class_name}\nConf: {confidence:.2f}',
                             color=color, fontsize=8)
            axes[i].axis('off')

    plt.suptitle(f'{model_name} Predictions', fontsize=14)
    plt.tight_layout()
    plt.show()

# Updated function calls
show_predictions(mlp_model, test_dataset, dataset_info, "MLP")
show_predictions(cnn_model, test_dataset, dataset_info, "CNN")

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS AND ANALYSIS")
print("="*80)

print("\n1. ACCURACY COMPARISON:")
print(f"   - CNN achieved {cnn_results['final_test_acc']:.2f}% accuracy")
print(f"   - MLP achieved {mlp_results['final_test_acc']:.2f}% accuracy")
print(f"   - CNN outperformed MLP by {cnn_results['final_test_acc'] - mlp_results['final_test_acc']:.2f} percentage points")

print("\n2. PARAMETER EFFICIENCY:")
mlp_params = sum(p.numel() for p in mlp_model.parameters())
cnn_params = sum(p.numel() for p in cnn_model.parameters())
print(f"   - MLP parameters: {mlp_params:,}")
print(f"   - CNN parameters: {cnn_params:,}")
print(f"   - CNN has {((cnn_params - mlp_params) / mlp_params * 100):+.1f}% parameters relative to MLP")

Step 4: Activate sweep agents

In [ ]:
wandb.agent(sweep_id, train, count=5)

TRIAL

In [ ]:
# ========================
# CIFAR-10 CNN + W&B Sweeps
# ========================

!pip install wandb -q

import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# ------------------------
# Sweep Configuration
# ------------------------
sweep_config = {
    'method': 'random',   # options: 'grid', 'random', 'bayes'
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'values': [0.01, 0.001, 0.0001]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'epochs': {
            'value': 5
        }
    }
}

# ------------------------
# CNN Model
# ------------------------
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# ------------------------
# Training Function
# ------------------------
def train():
    config = wandb.config

    # ----- DATA -----
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5))
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=config.batch_size, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=config.batch_size, shuffle=False)

    # ----- MODEL -----
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()

    if config.optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    else:
        optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9)

    # ----- TRAINING LOOP -----
    for epoch in range(config.epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)

        # ----- VALIDATION -----
        model.eval()
        correct, total, val_loss = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        avg_val_loss = val_loss / len(testloader)

        # ----- LOG METRICS -----
        wandb.log({
            "epoch": epoch,
            "train_loss": avg_loss,
            "val_loss": avg_val_loss,
            "val_acc": val_acc
        })

# ------------------------
# Run Sweep
# ------------------------
sweep_id = wandb.sweep(sweep_config, project="cifar10-cnn-sweep")
wandb.agent(sweep_id, function=train, count=5)  # count = number of runs


Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7a9d5d16fa40>> (for pre_run_cell):


AlreadyJoinedError: 

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 6yvv9lx4
Sweep URL: https://wandb.ai/jaydencruz824-belmont-university/cifar10-cnn-sweep/sweeps/6yvv9lx4


wandb: Agent Starting Run: jcuffx44 with config:
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/wandb/agents/pyagent.py", line 297, in _run_job
    self._function()
  File "/tmp/ipython-input-4163798001.py", line 128, in train
    wandb.log({
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 399, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 457, in wrapper_fn
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 444, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 2016, in log
    self._log(data

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7a9d5d16fa40>> (for post_run_cell):


AlreadyJoinedError: 